In [4]:
import json
import os
import shutil
from pathlib import Path
from collections import Counter

# ==============================================================================
# CONFIGURAÇÕES
# ==============================================================================
INPUT_DIR = Path('data/json/cleaned_factors')
OUTLIER_DIR = Path('data/json/Identified_outliers')
LIMITE_OUTLIER = 0  # Se aparecer x vezes ou menos, é considerado outlier

def analisar_e_segregar():
    # 1. Preparação do ambiente
    if not INPUT_DIR.exists():
        print(f"Erro: Diretório de entrada '{INPUT_DIR}' não encontrado.")
        return

    # Cria (ou limpa/recria se quiser ser agressivo, aqui apenas garante que existe)
    OUTLIER_DIR.mkdir(parents=True, exist_ok=True)

    arquivos_json = list(INPUT_DIR.glob("*.json"))
    print(f"📂 Lendo {len(arquivos_json)} arquivos para análise de frequência...")

    # ==========================================================================
    # PASSO 1: CONTAGEM GLOBAL (Mapear o território)
    # ==========================================================================
    contador_global = Counter()

    # Dicionário para guardar os fatores de cada arquivo em memória
    # (para evitar ler o disco duas vezes)
    cache_arquivos = {}

    for arquivo in arquivos_json:
        try:
            with open(arquivo, 'r', encoding='utf-8') as f:
                dados = json.load(f)

            fatores = dados.get('conteudo', {}).get('fatores_contribuintes', [])

            # Normalização básica (strip) para garantir contagem exata
            fatores = [f.strip() for f in fatores if f]

            if fatores:
                contador_global.update(fatores)
                cache_arquivos[arquivo] = fatores

        except Exception as e:
            print(f"⚠️ Erro ao ler {arquivo.name}: {e}")

    print(f"📊 Total de fatores únicos identificados: {len(contador_global)}")

    # ==========================================================================
    # PASSO 2: IDENTIFICAR OUTLIERS
    # ==========================================================================
    # Labels que aparecem LIMITE_OUTLIER vezes ou menos
    labels_outliers = {
        label for label, freq in contador_global.items()
        if freq <= LIMITE_OUTLIER
    }

    print(f"🔍 Identificados {len(labels_outliers)} labels considerados 'raros' (frequência <= {LIMITE_OUTLIER}).")

    # ==========================================================================
    # PASSO 3: SEGREGAR ARQUIVOS
    # ==========================================================================
    arquivos_movidos = 0

    print("\n📦 Iniciando segregação de arquivos...")

    for arquivo_path, fatores in cache_arquivos.items():
        # Verifica se este arquivo contém ALGUM fator que está na lista de outliers
        contem_outlier = any(fator in labels_outliers for fator in fatores)

        if contem_outlier:
            destino = OUTLIER_DIR / arquivo_path.name
            try:
                shutil.move(str(arquivo_path), str(destino))
                arquivos_movidos += 1
            except Exception as e:
                print(f"❌ Erro ao mover {arquivo_path.name}: {e}")

    # ==========================================================================
    # PASSO 4: RELATÓRIO FINAL
    # ==========================================================================
    print("\n" + "="*60)
    print(f"RELATÓRIO DE FATORES COMUNS (Frequência > {LIMITE_OUTLIER})")
    print("="*60)

    # Filtra apenas os comuns para exibição
    fatores_comuns = {
        label: freq for label, freq in contador_global.items()
        if freq > LIMITE_OUTLIER
    }

    # Ordena alfabeticamente pela chave (nome do fator)
    for fator, freq in sorted(fatores_comuns.items()):
        print(f"• {fator}: {freq}")

    print("\n" + "="*60)
    print("RESUMO DA OPERAÇÃO")
    print("="*60)
    print(f"📂 Arquivos analisados: {len(arquivos_json)}")
    print(f"📉 Arquivos segregados (Outliers): {arquivos_movidos}")
    print(f"✅ Arquivos mantidos (Dataset Limpo): {len(arquivos_json) - arquivos_movidos}")
    print(f"📁 Local dos outliers: {OUTLIER_DIR}")
    print("="*60)

if __name__ == "__main__":
    analisar_e_segregar()

📂 Lendo 1567 arquivos para análise de frequência...
📊 Total de fatores únicos identificados: 283
🔍 Identificados 0 labels considerados 'raros' (frequência <= 0).

📦 Iniciando segregação de arquivos...

RELATÓRIO DE FATORES COMUNS (Frequência > 0)
• - aplicação dos comandos: 1
• - manutenção da aeronave: 1
• - não foi possível identificar os fatores contribuintes desta ocorrência: 1
• ansiedade: 1
• aplicação de comando: 2
• aplicação de comandos: 33
• aplicação de comandos (contribuiu): 39
• aplicação de comandos (contribuiu) - julgamento de pilotagem (contribuiu) planejamento de voo (indeterminado): 1
• aplicação de comandos (indeterminado): 12
• aplicação de comandos - atitude - condições meteorológicas adversas: 1
• aplicação de comandos da aeronave: 1
• aplicação dos comando (contribuiu): 1
• aplicação dos comandos: 114
• aplicação dos comandos (contribuiu): 137
• aplicação dos comandos (contribuiu) - atitude (contribuiu): 1
• aplicação dos comandos (contribuiu) atitude (contribuiu